In [3]:
import pandas as pd
import requests

ModuleNotFoundError: No module named 'requests'

In [4]:
%pip install requests

  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_10_9_x86_64.whl (121 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests

requests.__version__

'2.31.0'